<a href="https://colab.research.google.com/github/ev1025/DA_Study/blob/main/%ED%81%AC%EB%A1%A4%EB%A7%81_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install webdriver_manager

In [ ]:
pip install selenium

In [ ]:
pip install BeautifulSoup4

# 카카오맵 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 크롬 옵션
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('--disable-gpu--')
options.add_argument('lang=ko_KR')

# 크롬 드라이버 설치
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 카카오맵 방문
driver.get("https://map.kakao.com/")

# 페이지 로드시간 암묵적 대기
driver.implicitly_wait(10)

# csv 입력받을 DataFrame
dog_map_df = pd.DataFrame(columns=('name','star','adr','category'))

# 검색 할 목록
search_list = ['서울 애견 미용실','서울 애견호텔', '서울 반려동물 미용실',\
                 '서울 동물병원', '서울 강아지 수제간식','서울 애견 수제간식', '서울 애견동반', '서울 강아지유치원','서울 애견유치원', '서울 애견용품']

# 목록에서 하나씩 검색
for search_id in search_list:
    # 검색상자 찾기
    search_box = driver.find_element(By.CSS_SELECTOR, "#search\.keyword\.query")
    # 검색할 단어 입력
    search_box.send_keys(f"{search_id}")
    time.sleep(1)
    # 검색버튼 누르기
    search_box.send_keys(Keys.ENTER)
    time.sleep(1)

    # 지도설정 팝업 제거(최초 1회만 나타나서 예외구문 적용)
    try:
        driver.find_element(By.CSS_SELECTOR, "body > div.coach_layer.coach_layer_type1 > div > div > div > span").click()
    except:
        pass
    time.sleep(1)

    # 최초에 페이지번호 뜨지 않고 더보기 눌러줘야함(최초 1회만 나타나서 예외구문 적용)
    try:
        more_btn = driver.find_element(By.CSS_SELECTOR, "#info\.search\.place\.more")
        more_btn.click()
    except:
        pass
    time.sleep(1)

    # dog_map_df에 입력받을 컬럼
    name = []
    star = []
    adr = []
    category = []         
    
    # 크롤링 시작
    # 페이지당 15개의 개체가 있다.
    # 이전, 페이지 1~5, 다음 버튼이 존재하며, 이전,다음버튼은 5개씩 넘어가게 되어있음
    
    # 다음버튼 누르는 횟수 개체 개수 = (15개 x 5page x 다음버튼 횟수)
    for p in range(16):
        # 1page ~ 5page 페이지이동 (6~10, 11~15 ...)
        # page버튼이 모자르면 종료(6~10구간에서 6,7,8까지만 있으면 8page에서 종료)
        for j in range(1,6):
            try:
                next_btn_1stp = driver.find_element(By.CSS_SELECTOR, f"#info\.search\.page\.no{j}")
                next_btn_1stp.click()
            except:
                break
            time.sleep(1)

            # 추출할 범위를 CSS로 선택해서 HTML형식으로 변환
            js_script = "document.querySelector('.placelist').innerHTML"
            # 자바스크립트형식 읽어들이기
            raw = driver.execute_script("return " + js_script)
            html = BeautifulSoup(raw, 'html.parser')
            time.sleep(2)

            # 직접적인 데이터가 있는 CSS그룹 선택하여 HTML형식을 list에 저장
            contents = html.find_all(class_='PlaceItem clickArea')
            time.sleep(2)

            # CSS그룹에서 필요한 값 추출
            for i in contents:
                cr_name = i.find(class_='link_name').text
                name.append(cr_name)

                try:
                    cr_star = float(i.select_one("em.num").text)
                    star.append(cr_star)
                except:
                    star.append(0)

                cr_adr = i.select_one('.addr').text.replace('\n', '').split(' ')
                if cr_adr[0] == '서울':
                    adr.append(cr_adr[1])

                cr_category = i.find(class_='subcategory clickable').text
                category.append(cr_category)
                

        try:
            next_btn = driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]')
            next_btn.send_keys(Keys.ENTER)
        except:
            print(f'{search_id}수집 끝')
            break

   
    # 추출한 데이터 입력
    search_data = pd.DataFrame({'name':name,'star':star,'adr':adr,'category':category})
    dog_map_df = pd.concat([dog_map_df,search_data]) 
  
    # 검색어 지우기
    driver.implicitly_wait(3)
    search_box.clear()
    

# 중복제거
dog_map_df.drop_duplicates(['name'], keep='first', inplace=True)

# csv파일로 저장
dog_map_df.to_csv(path_or_buf=f'./dog_map_data.csv',encoding='utf-8-sig', index=False)

# 크롬 웹페이지를 닫음
driver.close()

dog_map_df

### 탑텐몰 제품, 가격, 할인률

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd

driver = webdriver.Chrome("C:/Users/eg287/chromedriver") # 나의 웹드라이버의 위치(설정-도움말-버전에서 확인 후 최신 드라이버 사용)
driver.get("https://www.topten10mall.com/kr/front/search/categorySearch.do?ctgNo=37341") # 웹사이트 방문

# 팝업 창 제거
# driver.find_element(By.CSS_SELECTOR, "button#intro_popup_close").click()
driver.implicitly_wait(10) # 페이지 로드시간에 10초 암묵적 대기

# 검색창에 검색어 입력하기
search_box = driver.find_element(By.CSS_SELECTOR, "#searchWord")
search_box.send_keys("맨투맨")

time.sleep(3)

# 검색버튼 누르기
search_box.send_keys(Keys.ENTER)
# raw = driver.page_source # 현재 랜더링된 페이지의 정보를 가져옴

# 추출한 데이터 저장할 DataFrame 생성
empty = pd.DataFrame(columns=['name','price','discount'])
num = 0 # 추출한 값을 넣을 인덱스번호

# 크롤링
for p in range(28):
    # 5초 delay
    time.sleep(2)
    
    # 자바로 HTML가져오기(추출할 범위)
    js_script = "document.querySelector('#divList').innerHTML"
    raw = driver.execute_script("return " + js_script)
    
    # requests로 HTML 가져오기
    # raw = requests.get("https://www.topten10mall.com/kr/front/search/totalSearch.do?searchTerm=%EC%85%94%EC%B8%A0")

    # 파서로 추출할 대상 선택
    html = BeautifulSoup(raw, 'html.parser')
    contents = html.select('.card-goods__body')

    # 추출대상별 구체적으로 추출할 클래스 값 추출
    for i in contents:
        name = i.select_one('.card-goods__text').text
        try:
            price = i.select_one('.card-goods__price').text
        except:
            price = 'NULL'
        try:
            discount = i.select_one('.card-goods__discount').text
        except:
            discount = '0%'

        # empty 데이터프레임에 추출한 값 넣기
        empty_dict = {'name':name, 'price':price, 'discount':discount}
        for key,value in empty_dict.items():
            empty.loc[num,key] = value
        num += 1 # 인덱스 번호 1씩 증가하도록
        # print(name, price, discount)
        
        
    # 다음 페이지로 이동(다음페이지 없을때까지)
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, '#searchGoods > nav > ul > li:nth-last-child(1)')
        next_btn.click()
    except:
        print("데이터 수집 완료")
        break


# empty.to_csv(path_or_buf='./크롤링.csv',index=False, encoding = "utf-8-sig")
len(empty)

### 네이버 실패

In [ ]:
# 네이버 안돼
url = 'https://map.naver.com/v5/favorite/myPlace/folder/74b5e2268e0444c999227a4ec63e2839?c=10.53,0,0,0,dh'
html = requests.get(url)
soup = BeautifulSoup(html.content,'html.parser')
soup
# data = soup.select('.\_2gfp\-T')
# data

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd

driver = webdriver.Chrome("C:/Users/eg287/chromedriver") # 나의 웹드라이버의 위치(설정-도움말-버전에서 확인 후 최신 드라이버 사용)
driver.get("https://map.naver.com/v5/?c=15,0,0,0,dh")            # 웹사이트 방문


# 팝업 창 제거
# driver.find_element(By.CSS_SELECTOR, "button#intro_popup_close").click()
driver.implicitly_wait(10) # 페이지 로드시간에 10초 암묵적 대기

search_list = ['서울 동물병원']
# ,'서울 반려동물 미용실', '서울 강아지 간식', '서울 애견동반', '서울 강아지호텔', '서울 강아지유치원'
for search_id in search_list:

    # 검색창에 검색어 입력하기
    search_box = driver.find_element(By.CSS_SELECTOR, "div.input_box>input.input_search")
    search_box.send_keys(f"{search_id}")

    time.sleep(3)

    # 검색버튼 누르기
    search_box.send_keys(Keys.ENTER)

    name = []
    adr = []
    category = []


    # 크롤링
    for p in range(1):
            #5초 delay
            time.sleep(2)
            
            js_script = "document.querySelector('*').innerHTML"
            raw = driver.execute_script("return " + js_script)
            html = BeautifulSoup(raw, 'html.parser')
            
            contents = html.find_all(class_='VLTHu OW9LQ')

            for i in contents:
                name.append(i.select_one(" span.place_bluelink YwYLL").text)

                address = i.select_one('#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.qbGlu > div.ouxiq.icT4K > div > div > span > a > span.hClKF').text.replace('\n', '').split(' ')
                if address[0] =='서울':
                    adr.append(address[1])
                
                category.append(i.select_one('#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.qbGlu > div.ouxiq.icT4K > a:nth-child(1) > div > div > span.YzBgS').text)
            
            try:    
                next_btn = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(7) > svg")
                next_btn.click()
            except:
                print("데이터 수집 완료")
                break
        
    
    search_data = pd.DataFrame({'name':name,'adr':adr,'category':category})        
    search_data.to_csv(path_or_buf=f'./{search_id}.csv',encoding='utf-8-sig', index=False)
    
    # 검색어 지우기
    search_box.clear()



# 크롬 웹페이지를 닫음
# driver.close()